In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from sklearn.linear_model import LinearRegression
np.random.seed(0)

In [2]:
import keras 
import tensorflow as tf
from keras.datasets import cifar10
from keras.callbacks import History 
#using tensorflow backend

Using TensorFlow backend.


In [3]:
def mse_custom(y_true,y_predicted):
    mse_mod = tf.math.reduce_mean(tf.math.abs(tf.subtract(y_true[3:-3],y_predicted[3:-3])))
    return mse_mod

In [4]:
def new_cases_per_day(fecha_sintomas,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -graph: defaults to False. if True, makes graph of new cases per day 
    -ret: defaults to False. if True, returns days-cases per day  np.array casos_por_dia
    given the days of beginning of symptoms, processes to get np.array of cases per day and days since epidemic
    can graph and return this vector with the defaults explained before
    """
    counts = np.bincount(fecha_sintomas)
    ultima_fecha = max(fecha_sintomas)
    aux = range(ultima_fecha+1)
    casos_por_dia = np.vstack((aux,counts[aux])).T #(days since inicio_epidemia x (fecha_inicio_sintomas == days))
    if graph == True:
        plt.xlabel("Dia")
        plt.ylabel("Casos")
        plt.axvspan(ultima_fecha-10, ultima_fecha+3, facecolor='r', alpha=0.5,label="últimos 10 días")
        plt.plot(casos_por_dia[:,0],casos_por_dia[:,1],'-ob',label="casos hasta dia: "+str(ultima_fecha))
        plt.style.use('ggplot')
        plt.legend()
        #plt.savefig("casos_por_dia_bariloche.pdf")
    if ret == True:
        return casos_por_dia   

In [5]:
def cumulative_cases(fecha_sintomas,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -graph: defaults to False. if True, makes graph of new cases per day 
    -ret: defaults to False. if True, returns days-cumulative cases per day np array
    given the days of beginning of symptoms, processes to get np.array of cumulative cases per day and days since epidemic
    can graph and return this vector with the defaults explained before
    """
    ultima_fecha = max(fecha_sintomas)
    casos_por_dia = new_cases_per_day(fecha_sintomas,ret=True)
    casos_acumulados_por_dia = np.copy(casos_por_dia)
    casos_acumulados_por_dia[:,1] = np.cumsum(casos_por_dia[:,1]) 
    if graph == True:
        plt.xlabel("Dia")
        plt.ylabel("Casos acumulados")
        plt.axvspan(ultima_fecha-10, ultima_fecha+3, facecolor='r', alpha=0.5,label="últimos 10 días")
        plt.scatter(casos_acumulados_por_dia[:,0],casos_acumulados_por_dia[:,1],c='b',label="casos acumulados hasta dia: "+str(ultima_fecha))
        plt.legend()
        plt.style.use('ggplot')
        plt.savefig("casos_acumulados_bariloche.pdf")
    if ret == True:
        return casos_acumulados_por_dia   

In [6]:
def repratio_t_conv(fecha_sintomas,a,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -graph: defaults to False. if True, makes graph of rep ratio per day
    -ret: defaults to False. if True, returns reproductive ratio per day np array of (days,r_days) and
    the n_t vector which is used to calculate it, which is extension of new cases per day made as 
    necessary, by taking means and not linear regression. 
    given the days of beginning of symptoms, processes to get np.array of cumulative cases per day and days since epidemic
    can graph and return this vector with the defaults explained before for days (0,lastday+4)
    conventional version, which means no coef a,b,c,d,e. r_t valid for days >= 6. 
    """
    casos_por_dia = new_cases_per_day(fecha_sintomas,ret=True)
    n_t = np.copy(casos_por_dia) #here we store data + proyection for days t+4 
    prox_dia = max(fecha_sintomas) + 1
    #completing the values for the rest of the values of n_t
    aux = np.zeros(2)
    i = 0
    while i < 4: #completing until the day t+4 bc im gonna need it later for averaging in 7 days
        aux[0] = prox_dia + i
        aux[1] = np.mean(n_t[prox_dia-7:,1])
        n_t  = np.vstack((n_t,aux)) #extending n_t up to day t+1
        i= i +1
    #create storage and calculate values for r_t
    r_t = np.copy(n_t) #here we will store the rt
    r_t = r_t.astype(float)
    dias_aux = np.arange(0,prox_dia+3)
    for i in dias_aux[dias_aux>=6]:
        aux = a[0]*n_t[i-6,1] + a[1]*n_t[i-5,1] + a[2]*n_t[i-4,1] #denominator of the r_t expression
        if(aux==0):
            aux = 1 #the first cases in which the denominator is == 0
        r_t[i,1] = min(((a[3]*n_t[i-1,1] + a[4]*n_t[i,1] + a[5]*n_t[i+1,1])/(aux)),4) #the rest of the days with limit value
    if graph == True:
        plt.style.use('ggplot')
        plt.axvspan(prox_dia-10, prox_dia+3, facecolor='r', alpha=0.5,label="últimos 10 días")
        plt.scatter(r_t[6:prox_dia,0],r_t[6:prox_dia,1],c='b',label=r'$r_t$')
        plt.xlabel("Dia")
        plt.ylabel(r'$r_t$',fontsize=12)
        plt.legend()
        plt.savefig("rt_raw_casos.pdf")
    if ret == True:
        return n_t,r_t

In [7]:
def risk_diagram(fecha_sintomas,pop,a,rep_fun=repratio_t_conv,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -pop: number of inhabitants of the region/population of interest
    -rep_fun: function to calculate empirical reproductive ratio. defaults tp repratio_t_conv (not optimized)
    -graph: defaults to False. if True, makes graph of risk diagram
    -ret: defaults to False. if True, returns the components of risk diagram in two np.arrays, which are
     the attack ratio (days>=20) as well as the rep ratio averaged over 7 days (days>=20). 
    """
    #we proceed to calculate what makes a risk diagram then
    #first, we will calculate r_t average in 7 days
    prox_dia = max(fecha_sintomas) + 1
    n_t,r_t = rep_fun(fecha_sintomas,a,False,True)
    r_t_seven = np.zeros(prox_dia) #real values for index>=9
    a_t = np.zeros(prox_dia) #real values for index >= 13
    i = 9
    while i < prox_dia:
        r_t_seven[i] = np.mean(r_t[i-3:i+4,1])
        i = i+1
    i = 13
    while i < prox_dia:
        a_t[i] = np.sum(n_t[i-13:i+1,1]) 
        i = i+1
    a_t = a_t * (100000/pop)
    if graph == True:
        plt.plot(a_t[13:],r_t_seven[13:],'-or',markersize=5) #not at all fancy risk diagram,looks reasonable
        plt.xlabel(r'$A_{t}^{14}$')
        plt.ylabel(r'$R_{t}^{7}$')
        plt.style.use('ggplot')
        plt.title('Diagrama de riesgo Bariloche',fontsize=10)
        plt.savefig('riskdiagram_brc.pdf')
    if ret == True:
        return r_t_seven,a_t

In [8]:
def sum_last14(casos_por_dia):
    ac_por_dia = np.copy(casos_por_dia)
    for i in casos_por_dia[:,0]:
        index = np.copy(casos_por_dia[casos_por_dia[:,0]<i+1])
        index = index[i-13<=index[:,0]]
        index = index[:,1]
        ac_por_dia[i,1] = np.sum(index) 
    return ac_por_dia

In [9]:
def predict_cases(fecha_sintomas,pop,a,rep_fun=repratio_t_conv,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -pop: number of inhabitants of the region/population of interest
    -rep_fun: function to calculate empirical reproductive ratio. defaults tp repratio_t_conv (not optimized)
    -graph: defaults to False. if True, makes graph of risk diagram
    -ret: defaults to False. if True, returns the predictions and cases per day
    predicts cases per day after calculating the risk diagram, using the empirical reproductive
    ratio given by rep_fun
    """
    prox_dia = max(fecha_sintomas)+1
    casos_por_dia = new_cases_per_day(fecha_sintomas,ret=True)
    r_def,a_t = risk_diagram(fecha_sintomas,pop,a,rep_fun,ret=True)
    r_def = r_def[13:]
    a_t = a_t[13:]
    p_t = r_def * a_t
    valid_days = np.arange(19,prox_dia+6,1)
    casos_por_dia = sum_last14(casos_por_dia) #gets all active cases in the last 14-days 
    if graph == True:
        plt.style.use('ggplot')
        plt.xlabel('Dias desde el comienzo')
        plt.ylabel('Infectados activos en BRC')
        plt.plot(casos_por_dia[:,0],casos_por_dia[:,1]*(100000/pop),'-ob',markersize=4,label="casos hasta dia: "+str(prox_dia))
        plt.plot(valid_days,p_t,linewidth=3,label='prediccion')
        plt.legend()
        #plt.savefig('9oct_predic_casos.pdf')
    if ret == True:
        error_global = np.linalg.norm(casos_por_dia[19:,1]-p_t[:-6])
        return error_global

In [10]:
def change_dates(df):
    #changes fecha_inicio_sintomas according to new criteria
    filt_df1 = (df.fecha_inicio_sintomas.isnull()) #filter fecha_inicio_sintomas = inexistant
    df_sin_fecha = df.loc[filt_df1]
    n_size = df_sin_fecha.shape[0]
    df.loc[filt_df1,"fecha_inicio_sintomas"] = df.loc[filt_df1,"fecha_apertura"] - np.random.randint(0,9,n_size)
    filt_df2 = (df.fecha_inicio_sintomas < 0)
    df[filt_df2] = 0

In [11]:
data = pd.read_csv("casos/CasosCovid19_arg.txt",sep=",",quotechar='"',
                   parse_dates=["fecha_inicio_sintomas","fecha_apertura"],na_values=['']) #data loading
df = pd.DataFrame(data) #converting to dataframe for use

In [12]:
filt_df1 = (df.fecha_inicio_sintomas.notnull()) #filter fecha_inicio_sintomas = inexistant
inicio_epidemia = min(df.loc[filt_df1,"fecha_inicio_sintomas"]) #first symptoms of a person registered
ultima_actualizacion_sintomas = max(df.loc[filt_df1,"fecha_inicio_sintomas"]) #last day symptoms of a person registered
ultima_actualizacion_apertura = max(df["fecha_apertura"])
df["fecha_inicio_sintomas"] -= inicio_epidemia #correcting by inicio_epidemia 
df["fecha_apertura"] -= inicio_epidemia #correcting by inicio_epidemia
df.fecha_inicio_sintomas = df.fecha_inicio_sintomas.dt.days #change to int, ditch days 
df.fecha_apertura = df.fecha_apertura.dt.days #change to int, ditch days
change_dates(df) #replaces non existing fecha_inicio_sintomas acc to new criteria
print("primer sintoma de persona confirmada: ",inicio_epidemia)
print("ultimo sintoma de persona confirmada: ",ultima_actualizacion_sintomas)
print("ultima apertura de persona confirmada: ",ultima_actualizacion_apertura)

primer sintoma de persona confirmada:  2020-01-01 00:00:00
ultimo sintoma de persona confirmada:  2021-01-11 00:00:00
ultima apertura de persona confirmada:  2021-01-11 00:00:00


In [13]:
fecha_sintomas = df.fecha_inicio_sintomas.to_numpy() #numpy array of fecha_inicio_sintomas
fecha_apertura = df.fecha_apertura.to_numpy() #numpy array of fecha_apertura
fecha_sintomas = fecha_sintomas.astype(int)
fecha_sintomas = fecha_sintomas[fecha_sintomas <= max(fecha_sintomas) - 10]

In [14]:
aw = np.zeros(6) + 1
pop = 100000
rep_fun = repratio_t_conv
r_def,a_t = risk_diagram(fecha_sintomas,pop,aw,rep_fun,ret=True) #hasta aca perfecto
a_t = a_t[10:] 
a = a_t[:-4]

In [15]:
casos_por_dia = new_cases_per_day(fecha_sintomas,ret=True)
casos_por_dia = sum_last14(casos_por_dia)
casos_por_dia = casos_por_dia[17:,1]
b = np.zeros(casos_por_dia.shape[0]+3,dtype=float)
b[:casos_por_dia.shape[0]] = casos_por_dia

In [16]:
casos_por_dia = new_cases_per_day(fecha_sintomas,ret=True)
n_t = np.copy(casos_por_dia)
prox_dia = max(fecha_sintomas) + 1
#completing the values for the rest of the values of n_t
aux = np.zeros(2)
i = 0
while i < 4: #completing until the day t+4 bc im gonna need it later for averaging in 7 days
    aux[0] = prox_dia + i
    aux[1] = np.mean(n_t[prox_dia-7:,1])
    n_t  = np.vstack((n_t,aux)) #extending n_t up to day t+1
    i= i +1
dias = np.copy(n_t[:,0]).astype(int)
dias = dias[dias>=10]
dias = dias[dias<=max(fecha_sintomas)-4]
m1 = np.zeros((dias.shape[0],5)) #denominator of r7 expression
m2 = np.zeros((dias.shape[0],3)) #numerator of r7 expression

In [17]:
for i in dias:
    m1[i-10,:] = n_t[i-6:i-1,1]
    m2[i-10,:] = n_t[i-1:i+2,1]

In [18]:
n_dias = m1.shape[0]

In [19]:
def my_matrix(shape):
    m = np.zeros((shape,shape))
    i = 3
    while i < shape-3:
        m[i,i-3:i+4] = 1/7
        i = i + 1
    return m

In [20]:
def ret_matrix(shape):
    numpyA = my_matrix(shape)
    numpyA = keras.backend.variable(numpyA)
    mat = numpyA
    return mat

In [21]:
n_dias = m1.shape[0]

In [22]:
m1 += np.random.normal(0,25,m1.shape)
m2 += np.random.normal(0,25,m2.shape)

In [23]:
mat = ret_matrix(n_dias)

In [24]:
pop = 45376763 #poblacion en cuestion

In [ ]:
input_1 = keras.layers.Input(shape=(5,))
input_2 = keras.layers.Input(shape=(3,))
input_3 = keras.layers.Input(shape=(1,))
initializer = keras.initializers.Ones() #the vector of weights a
l_input_1 = keras.layers.Dense(1,activation='relu',kernel_initializer=initializer,use_bias=False)(input_1)
l_input_2 = keras.layers.Dense(1,activation='relu',use_bias=False,kernel_initializer=initializer,trainable=False)(input_2)
division = keras.layers.Lambda(lambda inputs:  tf.where(inputs[0] != 0, tf.math.minimum(inputs[1]/inputs[0],4), inputs[1]))([l_input_1, l_input_2])#calculates empirical rt
mean = keras.layers.Lambda(lambda x: keras.backend.dot(mat,x))(division) #calculates mean r_seven in seven days
cases = keras.layers.Multiply()([input_3,mean]) #cases for valid prediction,net output, padded with zeros at the end
model = keras.Model(inputs=[input_1,input_2,input_3],outputs=[cases])
optimizer = keras.optimizers.Adam(1e-3)
model.compile(optimizer, loss=mse_custom)
history = model.fit([m1,m2,a*(100000/pop)], b*(100000/pop), epochs=30000, batch_size=n_dias,shuffle=False)
model.summary()

Epoch 1/30000
353/353 [==============================] - 0s 932us/step - loss: 59.2874
Epoch 2/30000
353/353 [==============================] - 0s 8us/step - loss: 59.2019
Epoch 3/30000
353/353 [==============================] - 0s 8us/step - loss: 59.1161
Epoch 4/30000
353/353 [==============================] - 0s 6us/step - loss: 59.0302
Epoch 5/30000
353/353 [==============================] - 0s 6us/step - loss: 58.9441
Epoch 6/30000
353/353 [==============================] - 0s 8us/step - loss: 58.8579
Epoch 7/30000
353/353 [==============================] - 0s 8us/step - loss: 58.7714
Epoch 8/30000
353/353 [==============================] - 0s 6us/step - loss: 58.6848
Epoch 9/30000
353/353 [==============================] - 0s 6us/step - loss: 58.5979
Epoch 10/30000
353/353 [==============================] - 0s 6us/step - loss: 58.5109
Epoch 11/30000
353/353 [==============================] - 0s 8us/step - loss: 58.4236
Epoch 12/30000
353/353 [==============================] - 0s 

Epoch 97/30000
353/353 [==============================] - 0s 6us/step - loss: 49.8685
Epoch 98/30000
353/353 [==============================] - 0s 6us/step - loss: 49.7540
Epoch 99/30000
353/353 [==============================] - 0s 8us/step - loss: 49.6392
Epoch 100/30000
353/353 [==============================] - 0s 6us/step - loss: 49.5240
Epoch 101/30000
353/353 [==============================] - 0s 8us/step - loss: 49.4083
Epoch 102/30000
353/353 [==============================] - 0s 8us/step - loss: 49.2923
Epoch 103/30000
353/353 [==============================] - 0s 6us/step - loss: 49.1758
Epoch 104/30000
353/353 [==============================] - 0s 6us/step - loss: 49.0590
Epoch 105/30000
353/353 [==============================] - 0s 6us/step - loss: 48.9416
Epoch 106/30000
353/353 [==============================] - 0s 6us/step - loss: 48.8239
Epoch 107/30000
353/353 [==============================] - 0s 6us/step - loss: 48.7058
Epoch 108/30000
353/353 [=====================

Epoch 192/30000
353/353 [==============================] - 0s 6us/step - loss: 36.7997
Epoch 193/30000
353/353 [==============================] - 0s 6us/step - loss: 36.6332
Epoch 194/30000
353/353 [==============================] - 0s 6us/step - loss: 36.4659
Epoch 195/30000
353/353 [==============================] - 0s 6us/step - loss: 36.2979
Epoch 196/30000
353/353 [==============================] - 0s 6us/step - loss: 36.1291
Epoch 197/30000
353/353 [==============================] - 0s 6us/step - loss: 35.9596
Epoch 198/30000
353/353 [==============================] - 0s 6us/step - loss: 35.7893
Epoch 199/30000
353/353 [==============================] - 0s 6us/step - loss: 35.6182
Epoch 200/30000
353/353 [==============================] - 0s 6us/step - loss: 35.4464
Epoch 201/30000
353/353 [==============================] - 0s 3us/step - loss: 35.2738
Epoch 202/30000
353/353 [==============================] - 0s 6us/step - loss: 35.1004
Epoch 203/30000
353/353 [==================

Epoch 287/30000
353/353 [==============================] - 0s 6us/step - loss: 16.7079
Epoch 288/30000
353/353 [==============================] - 0s 6us/step - loss: 16.4443
Epoch 289/30000
353/353 [==============================] - 0s 6us/step - loss: 16.1812
Epoch 290/30000
353/353 [==============================] - 0s 6us/step - loss: 15.9196
Epoch 291/30000
353/353 [==============================] - 0s 8us/step - loss: 15.6574
Epoch 292/30000
353/353 [==============================] - 0s 5us/step - loss: 15.3955
Epoch 293/30000
353/353 [==============================] - 0s 6us/step - loss: 15.1341
Epoch 294/30000
353/353 [==============================] - 0s 11us/step - loss: 14.8718
Epoch 295/30000
353/353 [==============================] - 0s 6us/step - loss: 14.6085
Epoch 296/30000
353/353 [==============================] - 0s 6us/step - loss: 14.3442
Epoch 297/30000
353/353 [==============================] - 0s 6us/step - loss: 14.0789
Epoch 298/30000
353/353 [=================

Epoch 382/30000
353/353 [==============================] - 0s 6us/step - loss: 6.4258
Epoch 383/30000
353/353 [==============================] - 0s 6us/step - loss: 6.4256
Epoch 384/30000
353/353 [==============================] - 0s 6us/step - loss: 6.4254
Epoch 385/30000
353/353 [==============================] - 0s 6us/step - loss: 6.4251
Epoch 386/30000
353/353 [==============================] - 0s 6us/step - loss: 6.4248
Epoch 387/30000
353/353 [==============================] - 0s 6us/step - loss: 6.4245
Epoch 388/30000
353/353 [==============================] - 0s 6us/step - loss: 6.4241
Epoch 389/30000
353/353 [==============================] - 0s 8us/step - loss: 6.4238
Epoch 390/30000
353/353 [==============================] - 0s 6us/step - loss: 6.4234
Epoch 391/30000
353/353 [==============================] - 0s 6us/step - loss: 6.4230
Epoch 392/30000
353/353 [==============================] - 0s 6us/step - loss: 6.4225
Epoch 393/30000
353/353 [=============================

...

Epoch 21408/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6992
Epoch 21409/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6992
Epoch 21410/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6992
Epoch 21411/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6992
Epoch 21412/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6994
Epoch 21413/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6997
Epoch 21414/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6993
Epoch 21415/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6991
Epoch 21416/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6999
Epoch 21417/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6995
Epoch 21418/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6995
Epoch 21419/30000
353/353 [=====

Epoch 21502/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6991
Epoch 21503/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6989
Epoch 21504/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6993
Epoch 21505/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6990
Epoch 21506/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6993
Epoch 21507/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6995
Epoch 21508/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6991
Epoch 21509/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6992
Epoch 21510/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6994
Epoch 21511/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6988
Epoch 21512/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6990
Epoch 21513/30000
353/353 [=====

353/353 [==============================] - 0s 6us/step - loss: 4.6986
Epoch 21596/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6986
Epoch 21597/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6986
Epoch 21598/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6985
Epoch 21599/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6985
Epoch 21600/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6986
Epoch 21601/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6986
Epoch 21602/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6988
Epoch 21603/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6986
Epoch 21604/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6986
Epoch 21605/30000
353/353 [==============================] - 0s 5us/step - loss: 4.6985
Epoch 21606/30000
353/353 [=======================

Epoch 21689/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6985
Epoch 21690/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6985
Epoch 21691/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6982
Epoch 21692/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6990
Epoch 21693/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6988
Epoch 21694/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6983
Epoch 21695/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6988
Epoch 21696/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6987
Epoch 21697/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6982
Epoch 21698/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6991
Epoch 21699/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6991
Epoch 21700/30000
353/353 [=====

353/353 [==============================] - 0s 6us/step - loss: 4.6980
Epoch 21783/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6979
Epoch 21784/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6980
Epoch 21785/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6980
Epoch 21786/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6979
Epoch 21787/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6979
Epoch 21788/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6979
Epoch 21789/30000
353/353 [==============================] - 0s 11us/step - loss: 4.6980
Epoch 21790/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6980
Epoch 21791/30000
353/353 [==============================] - 0s 6us/step - loss: 4.6979
Epoch 21792/30000
353/353 [==============================] - 0s 8us/step - loss: 4.6979
Epoch 21793/30000
353/353 [======================

In [ ]:
res = model.predict([m1,m2,a],batch_size=n_dias)

In [ ]:
first_layer_weights = model.layers[2].get_weights()
second_layer_weights = model.layers[3].get_weights()
print(first_layer_weights,second_layer_weights)

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
res = model.predict([m1,m2,a],batch_size=n_dias)

In [ ]:
plt.plot(res[3:-3],label='result')
plt.plot(b[3:-3],label='groundtruth')
plt.legend()